In [1]:
# -----------------------------
# FIXED LONG-TABLE CONSTRUCTION
# -----------------------------
import pandas as pd

type_map = [
    ('Tech',    'Technical'),
    ('Attacks', 'Atack'),      # <-- IMPORTANT: actual CSV prefix is "Atack", not "Attack"
    ('Data',    'Data'),
    ('Scams',   'Scams'),
    ('Cash',    'Cash'),
]

records = []

for _, row in df.iterrows():
    rid = row['ResponseID']
    for out_label, prefix in type_map:
        for pos in range(1, 6):

            # Country name
            country = row.get(f'{prefix}{pos}')
            if pd.isna(country):
                continue

            country_str = str(country).strip()
            if country_str in ['', '--']:
                continue

            # Raw scores for this nomination
            impact = row.get(f'{prefix}{pos}_impact')
            prof   = row.get(f'{prefix}{pos}_professional')
            tech   = row.get(f'{prefix}{pos}_techskill')

            # *** STRICT FILTER ***
            # The WCI authors drop any nomination missing ANY of the 3 scores.
            if pd.isna(impact) or pd.isna(prof) or pd.isna(tech):
                continue

            # Append clean nomination row
            records.append({
                'ResponseID': rid,
                'Country': country_str,
                'CrimeType': out_label,
                'Impact': float(impact),
                'Professionalism': float(prof),
                'TechSkill': float(tech),
            })

long_df = pd.DataFrame.from_records(records)

print("Long table shape (after strict filtering):", long_df.shape)
display(long_df.head())

# -----------------------------
# PER-NOMINATION MEAN SCORE
# -----------------------------
long_df['NomScore'] = long_df[['Impact', 'Professionalism', 'TechSkill']].mean(axis=1)

# -----------------------------
# AGGREGATE: CountryScore_type and nominations_type
# -----------------------------
grp = long_df.groupby(['Country', 'CrimeType'])
agg = grp['NomScore'].agg(['mean', 'count']).reset_index()
agg = agg.rename(columns={
    'mean': 'CountryScore_type',
    'count': 'nominations_type'
})

# -----------------------------
# PER-TYPE WCI (Eq. 2)
# -----------------------------
N_RESPONDENTS_PUBLISHED = 92
agg['WCI_type'] = agg['CountryScore_type'] * (agg['nominations_type'] / N_RESPONDENTS_PUBLISHED) * 10

# Pivot to wide layout
per_type = agg.pivot_table(
    index='Country',
    columns='CrimeType',
    values=['CountryScore_type', 'nominations_type', 'WCI_type'],
    aggfunc='first'
)
per_type.columns = [f"{a}__{b}" for a, b in per_type.columns]
per_type = per_type.reset_index()

# Ensure all five types exist (fill missing with 0)
for t in ['Tech', 'Attacks', 'Data', 'Scams', 'Cash']:
    for base in ['CountryScore_type', 'nominations_type', 'WCI_type']:
        col = f'{base}__{t}'
        if col not in per_type.columns:
            per_type[col] = 0.0

# -----------------------------
# AVG RAW TYPE SCORE
# -----------------------------
per_type['AvgTypeScore'] = (
    per_type['CountryScore_type__Tech'] +
    per_type['CountryScore_type__Attacks'] +
    per_type['CountryScore_type__Data'] +
    per_type['CountryScore_type__Scams'] +
    per_type['CountryScore_type__Cash']
) / 5

# -----------------------------
# TOTAL NOMS ACROSS ALL TYPES
# -----------------------------
per_type['TotalNominations'] = (
    per_type['nominations_type__Tech'] +
    per_type['nominations_type__Attacks'] +
    per_type['nominations_type__Data'] +
    per_type['nominations_type__Scams'] +
    per_type['nominations_type__Cash']
)

# -----------------------------
# OVERALL WCI (Eq. 3)
# -----------------------------
DENOM_ALL = 92 * 5  # = 460
per_type['WCI Score'] = per_type['AvgTypeScore'] * (per_type['TotalNominations'] / DENOM_ALL) * 10

# -----------------------------
# OVERALL I / P / TS
# -----------------------------
ipt = long_df.groupby('Country')[['Impact', 'Professionalism', 'TechSkill']].mean().reset_index()
ipt = ipt.rename(columns={'Impact': 'I', 'Professionalism': 'P', 'TechSkill': 'TS'})

# -----------------------------
# FINAL OUTPUT TABLE
# -----------------------------
final = per_type.merge(ipt, on='Country', how='left')

final['Tech']    = final['WCI_type__Tech']
final['Attacks'] = final['WCI_type__Attacks']
final['Data']    = final['WCI_type__Data']
final['Scams']   = final['WCI_type__Scams']
final['Cash']    = final['WCI_type__Cash']

final = final[['Country', 'I', 'P', 'TS', 'WCI Score', 'Tech', 'Attacks', 'Data', 'Scams', 'Cash']]
final = final.sort_values('WCI Score', ascending=False).reset_index(drop=True)
final.insert(0, 'Rank', final.index + 1)

display(final.head(15))

NameError: name 'df' is not defined